In [67]:
#imports
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix,recall_score,precision_score,f1_score

In [45]:
#read data file with all default calumns
train_df=pd.read_csv('dataset.csv',index_col='customerID')

In [46]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 7590-VHVEG to 3186-AJIEK
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   object 
 3   Dependents        7043 non-null   object 
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   object 
 6   MultipleLines     7043 non-null   object 
 7   InternetService   7043 non-null   object 
 8   StreamingService  7043 non-null   object 
 9   Contract          7043 non-null   object 
 10  PaperlessBilling  7043 non-null   object 
 11  PaymentMethod     7043 non-null   object 
 12  MonthlyCharges    7043 non-null   float64
 13  TotalCharges      7043 non-null   object 
 14  Churn             7043 non-null   object 
dtypes: float64(1), int64(2), object(12)
memory usage: 880.4+ KB


In [47]:
#Apart from tenure,MonthlyCharges,TotalCharges all are categorical columns
train_df[train_df['TotalCharges']==' ']

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,StreamingService,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
customerID,,,,,,,,,,,,,,,
4472-LVYGI,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,Two year,Yes,Bank transfer (automatic),52.55,,No
3115-CZMZD,Male,0,No,Yes,0,Yes,No,No,No internet service,Two year,No,Mailed check,20.25,,No
5709-LVOEQ,Female,0,Yes,Yes,0,Yes,No,DSL,Yes,Two year,No,Mailed check,80.85,,No
4367-NUYAO,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,Two year,No,Mailed check,25.75,,No
1371-DWPAZ,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,Two year,No,Credit card (automatic),56.05,,No
7644-OMVMY,Male,0,Yes,Yes,0,Yes,No,No,No internet service,Two year,No,Mailed check,19.85,,No
3213-VVOLG,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,Two year,No,Mailed check,25.35,,No
2520-SGTTA,Female,0,Yes,Yes,0,Yes,No,No,No internet service,Two year,No,Mailed check,20.00,,No
2923-ARZLG,Male,0,Yes,Yes,0,Yes,No,No,No internet service,One year,Yes,Mailed check,19.70,,No


In [48]:
train_df['TotalCharges'] = np.where(train_df['TotalCharges'] == ' ', train_df['MonthlyCharges'], train_df['TotalCharges'])

In [49]:
X=train_df.loc[:,:'TotalCharges']
y=train_df['Churn'].ravel()
y=np.where(y == 'Yes', 1, 0)

In [50]:
#train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(5634, 14) (5634,)
(1409, 14) (1409,)


In [51]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 7590-VHVEG to 3186-AJIEK
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   object 
 3   Dependents        7043 non-null   object 
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   object 
 6   MultipleLines     7043 non-null   object 
 7   InternetService   7043 non-null   object 
 8   StreamingService  7043 non-null   object 
 9   Contract          7043 non-null   object 
 10  PaperlessBilling  7043 non-null   object 
 11  PaymentMethod     7043 non-null   object 
 12  MonthlyCharges    7043 non-null   float64
 13  TotalCharges      7043 non-null   object 
 14  Churn             7043 non-null   object 
dtypes: float64(1), int64(2), object(12)
memory usage: 880.4+ KB


In [52]:
cat_columns=[0,1,2,3,5,6,7,8,9,10,11,]

In [53]:
clf = CatBoostClassifier(
    iterations=100, 
    learning_rate=0.01, 
    #loss_function='CrossEntropy'
)

clf.fit(X_train,y_train, 
        cat_features=cat_columns
)

0:	learn: 0.6895292	total: 53.8ms	remaining: 5.33s
1:	learn: 0.6857925	total: 111ms	remaining: 5.42s
2:	learn: 0.6821208	total: 163ms	remaining: 5.28s
3:	learn: 0.6784746	total: 206ms	remaining: 4.95s
4:	learn: 0.6749787	total: 264ms	remaining: 5.02s
5:	learn: 0.6715154	total: 318ms	remaining: 4.98s
6:	learn: 0.6681576	total: 369ms	remaining: 4.91s
7:	learn: 0.6649298	total: 417ms	remaining: 4.8s
8:	learn: 0.6616967	total: 466ms	remaining: 4.72s
9:	learn: 0.6584171	total: 514ms	remaining: 4.62s
10:	learn: 0.6550197	total: 569ms	remaining: 4.61s
11:	learn: 0.6518830	total: 617ms	remaining: 4.53s
12:	learn: 0.6486508	total: 666ms	remaining: 4.46s
13:	learn: 0.6456293	total: 721ms	remaining: 4.43s
14:	learn: 0.6426512	total: 774ms	remaining: 4.38s
15:	learn: 0.6396092	total: 834ms	remaining: 4.38s
16:	learn: 0.6366876	total: 903ms	remaining: 4.41s
17:	learn: 0.6337572	total: 982ms	remaining: 4.47s
18:	learn: 0.6309590	total: 1.04s	remaining: 4.42s
19:	learn: 0.6281429	total: 1.1s	remainin

In [59]:
y_pred=clf.predict(X_test)

In [61]:
confusion_matrix(y_test, y_pred)

array([[961,  80],
       [218, 150]], dtype=int64)

In [72]:
print(roc_auc_score(y_test, y_pred))
print(recall_score(y_pred, y_test))
print(precision_score(y_test, y_pred))
print(f1_score(y_test, y_pred))

0.6653797560873741
0.6521739130434783
0.6521739130434783
0.5016722408026757
